TODO
1. 评估数据集与评估方法 这个最关键
2. 医学知识库
2. 百川模型多角色 / Prompt优化
3. embedding调优
4. 上下文扩宽
5. 不同百川模型参数规模性能比较


https://github.com/zhangsheng93/cMedQA2


In [4]:
# !pip install bitsandbytes -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.4 MB/s eta 0:00:00:00:0100:01


In [3]:
# import datasets
# # part 1
# knowledge_graph_dataset = datasets.load_dataset('FreedomIntelligence/huatuo_knowledge_graph_qa')
# # part 2
# encyclopedia_dataset = datasets.load_dataset('FreedomIntelligence/huatuo_encyclopedia_qa')
# # part 3 (only url)
# consultation_dataset = datasets.load_dataset('FreedomIntelligence/huatuo_consultation_qa')
# # testdatasets (6k)
# huatuo_testdatasets = datasets.load_dataset('FreedomIntelligence/huatuo26M-testdatasets')

# Load Language Model

In [6]:
import json
import torch
import pandas as pd
import numpy as np
import faiss
import pickle

# import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig

In [14]:
model = AutoModelForCausalLM.from_pretrained("./model/Baichuan2-7B-Chat-4bits", device_map="auto", trust_remote_code=True)
model.generation_config = GenerationConfig.from_pretrained(
       "./model/Baichuan2-7B-Chat-4bits"
    )
tokenizer = AutoTokenizer.from_pretrained("./model/Baichuan2-7B-Chat-4bits", trust_remote_code=True)
model = model.eval()


#  print("init model ...")
#     model = AutoModelForCausalLM.from_pretrained(
#         "baichuan-inc/Baichuan2-13B-Chat",
#         torch_dtype=torch.float16,
#         device_map="auto",
#         trust_remote_code=True
#     )
#     model.generation_config = GenerationConfig.from_pretrained(
#         "baichuan-inc/Baichuan2-13B-Chat"
#     )
#     tokenizer = AutoTokenizer.from_pretrained(
#         "baichuan-inc/Baichuan2-13B-Chat",
#         use_fast=False,
#         trust_remote_code=True
#     )
#     return model, tokenizer

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


# Load embedding model

可以参考 https://github.com/shibing624/text2vec 下载最新版本并尝试fine tune

In [3]:
from text2vec import SentenceModel
from text2vec import CosentModel, SentenceBertModel, BertMatchModel
from text2vec import cos_sim, compute_spearmanr, EncoderType, load_text_matching_test_data
t2v_model = SentenceModel('./model/t2v')

2023-10-21 13:25:30.541 | DEBUG    | text2vec.sentence_model:__init__:80 - Use device: cuda


# Util library

In [4]:
def dump_obj(obj_name, obj):
    with open('./data/tmp/' + obj_name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def restore_obj(obj_name):
    with open('./data/tmp/' + obj_name + '.pkl', 'rb') as f:
        obj = pickle.load(f)
        return obj
def to_embeddings(model, text):
    sentence_embeddings = model.encode(text)
    return sentence_embeddings
def calc_similarity_scores(model, sents1, sents2):
    # t1 = time.time()
    e1 = model.encode(sents1)
    e2 = model.encode(sents2)
    # spend_time = time.time() - t1
    s = cos_sim(e1, e2)
    sims = []
    for i in range(len(sents1)):
        sims.append(s[i][i])
    sims = np.array(sims)
    return sims

# Construct Knwoeldge base

https://github.com/FreedomIntelligence/Huatuo-26M

In [6]:
df_knowledge = pd.read_csv('./data/huatuo_knowledge.csv')
df_encyclopedia = pd.read_csv('./data/huatuo_encyclopedia.csv')
df_kb=pd.concat([df_knowledge, df_encyclopedia]).reset_index()

kb_dict = {}
for index, row in df_kb.iterrows():
    kb_dict[index] = {
        'q': row['questions'],
        'a': row['answers']
    }

In [10]:
dump_obj('kb_dict',kb_dict)

In [7]:
kb_dict=restore_obj('kb_dict')

# Construct Vector DB

In [67]:


data = to_embeddings(t2v_model,list(df_kb.questions))
data = data.astype(np.float32)

dim = 384
# Create index using index factory
index = faiss.index_factory(dim, "IVF2048,PQ32") 

# Training is required for some indexes like IVF
index.train(data) 

# Add vectors to index
index.add(data)
faiss.write_index(index, "index_file.index") #将index保存为index_file.index文件
index = faiss.read_index("index_file.index") #读入index_file.index文件

In [8]:
index = faiss.read_index("index_file.index") #读入index_file.index文件

# Query

In [29]:
def get_context(query):

    qa_id_list = get_nn_by_query(query)
    prompt = "你是一名临床医生，请参考以下回答资料回答问题，如果段落内容不相关就返回未查到相关信息。"
    max_length = 1500
    i = 0 
    context = prompt

    while i < len (qa_id_list) and len(context) < max_length:
        
        qid = i + 1
        q = kb_dict[qa_id_list[i]]['q']
        a = kb_dict[qa_id_list[i]]['a']
        tmp = context + "\n" + f"{qid}. 问题：{q} || 答案：{a}"
        if len(tmp) <= max_length:
            context = tmp
        else:
            break
        i = i + 1

    context = context + "\n" + "问题：" + query + " || 答案："
    return context

def get_nn_by_query(query):
    tv = to_embeddings(t2v_model, [query])
    dis, ind = index.search(tv, 10)
    return list(ind[0])

def query_model(query):
    context = get_context(query)
    inputs = tokenizer(context, return_tensors='pt')
    inputs = inputs.to('cuda:0')
    pred = model.generate(**inputs, max_new_tokens=1280, repetition_penalty=1.1)
    response = tokenizer.decode(pred.cpu()[0], skip_special_tokens=True)
    # print(response)

    return response[len(context):]

def ask_glm(text):
    response, history = model.chat(tokenizer, text)
    return response,history

In [39]:
# query = t
# context = get_context(query)
# inputs = tokenizer(context, return_tensors='pt')
# inputs = inputs.to('cuda:0')
# pred = model.generate(**inputs, max_new_tokens=128, repetition_penalty=1.1)

In [42]:
# tokenizer.decode(pred.cpu()[0], skip_special_tokens=True)

'你是一名临床医生，请参考以下回答资料回答问题，如果段落内容不相关就返回未查到相关信息。\n1. 问题：胃不舒服怎么办 || 答案：胃不舒服要在饮食方面我们要养成良好的饮食习惯，胃是人体的重要器官，我们每天都要靠它来分解消化食物。正因为它作为人体器官的重要性，我们也经常用到它，总会造成伤害，那么胃不舒服怎么办呢，每天三餐要按时的吃，当然早餐是肯定要吃，如果不吃早餐会对胃造成伤害的。在太热的时候不要吃一些比较冷的食物，会对胃造成很大的影响，胃会承受不了，这样会加重胃的负担，使胃更加地不舒服。我们可以适当的吃一些养胃护胃的碱性的的食物。如土豆、胡萝卜、芋头、葡萄酒、海带等等，有利于中和胃酸。每次不要吃得太饱，这样加重胃部的负担，长期下来易引发胃病。适当地饮用酸牛奶，利于恢复肠道内正常功能，有利于消除胃肠胀气。豆类以及豆制品等不适宜过多进食，易产生胃胀气，导致胃不舒服。如果出现胃不舒服的这种现象，我们可以跪在地上，从膝盖到脚趾都要接触到地面，伸直上半身，双手摆放自然，臀部缓慢往脚踝上压，在这过程中保持正常呼吸，维持一分钟该姿势，适当放松后，再将上半身向前倾，重复该动作5次。这样的目的在于消除胃肠胀气，还有助于减瘦大腿肌肉。而且还可以做些其他事去分散自己的注意力，胃不舒服的人应该要保持良好的情绪，精神要开朗，不要一直在想很痛很痛，痛死了，如果你一直这样想，那么你就会一直度那么痛苦，相反，如果你再暗示自己，忍一下就好了，忍一下就过去了，那样效果更佳。另外保证睡眠充足和注意保暖胃部，睡觉的时候要把被子盖在胃部上，以免寒气的加重，从而导致胃不舒服。\n2. 问题：胃痛怎么办 || 答案：出现胃痛的情况，可以腹部热敷，适量喝点温开水，一般能够有效缓解，也可以自我按摩穴位，比如内关穴、足三里。如果不能缓解，建议还是要去医院就诊。短时间的突然性的胃痛，大多是由于饮食不当、腹部受凉等原因引起的比如使用生冷食物以及过于辛辣的食物会引起胃痛。但是如果频繁的出现胃痛，或者是长时间剧烈的胃痛，要排除胃部的器质性病变导致的可能，需要去消化内科进一步就诊。所谓的胃痛实际上指的是位于上腹部剑突下方和肚脐以上这个区域，一般会认为这个部位的疼痛就是胃痛的表现，但实际上导致上腹部区域疼痛的器官不仅包括胃，还包括十二指肠、胆道系统、肝脏、胰腺等。因此对于胃痛的治疗不能仅仅是头痛医头、脚痛医脚，或者是通过喝热水

## 直接使用Baichuan回答，不补充知识。

In [42]:
prompt = """
你是一名医学生，正在参加考试。题目为单项选择题。参考下面例子，给出数字编号答案。
题目：书写药历是药师进行规范化药学服务的一项工作，下列内容一般不作为药历内容的是 || 选项: 1. 患者基本信息 2. 用药评价 3. 临床诊断要点 4. 用药记录 5. 患者知情同意书 || 答案: 5
题目：研究TDM的临床意义不包括 || 选项: 1. 监督临床用药 2. 研究药物在体内的代谢变化 3. 研究治疗无效的原因 4. 确定患者是否按医嘱服药 5. 研究合并用药的影响 || 答案: 3
题目："""
question = prompt+ '临床上药物可以配伍使用或者联合使用，若使用不当，可能出现配伍禁忌。下列药物配伍或者联合使用中，不合理的是 || 选项：1. 磺胺甲噁唑与甲氧苄啶联合应用 2. 地西泮注射液与0.9%氯化钠注射液混合滴注 3. 硫酸亚铁片与维生素C片同时使用 4. 阿莫西林与克拉维酸联合使用 5. 氨苄西林溶于5%葡萄糖注射液后在4小时内滴注 || 答案: '
# inputs = tokenizer(prompt+ '临床上药物可以配伍使用或者联合使用，若使用不当，可能出现配伍禁忌。下列药物配伍或者联合使用中，不合理的是 || 选项：1. 磺胺甲噁唑与甲氧苄啶联合应用 2. 地西泮注射液与0.9%氯化钠注射液混合滴注 3. 硫酸亚铁片与维生素C片同时使用 4. 阿莫西林与克拉维酸联合使用 5. 氨苄西林溶于5%葡萄糖注射液后在4小时内滴注 || 答案:', return_tensors='pt')
# inputs = inputs.to('cuda:0')
# pred = model.generate(**inputs, max_new_tokens=64, repetition_penalty=1.1)
# print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))
query_model(question)

'2'

## 问题答案数据集

In [31]:
import json 
questions = []
with open('./data/NLPEC/train.json') as f:
  for line in f:
    # Remove newline character
    line = line.strip()
    
    # Load the JSON data
    data = json.loads(line)
    
    # Now you can access the data
    # print(data['key1']) 
    q = data['questionText']
    o = data['option']
    a = data['answer']
    questions.append((q,o,a))

In [33]:
len(questions)

18703


# 问题从第三个开始，因为前两个用作了few shot 例子

In [45]:
# for item in tqdm(questions):
  # Do some work
from tqdm import tqdm

result = []
for q, o, a in tqdm(questions):
# for q, o, a in questions:
    question = prompt + q.strip() + ' || 选项：'
    i = 1
    for opt in o:
        question = question + str(i) + '. ' + opt + ' '
        i = i +1
    question = question + '|| 答案: '
    # print(question)
    sa = query_model(question)
    # print(sa)
    result.append((q, o, a, sa))

    # print(q, o, a)

100%|██████████| 18703/18703 [42:00<00:00,  7.42it/s]  


In [50]:
result[0][2][0],result[0][3]

('3', '3')

In [56]:
def accuracy(result):
    return len([i for i in result if i[2][0] == i[3]]) / len(result)

In [57]:
 accuracy(result)

0.3687109019943325

In [62]:
[i for i in result if i[2][0] != i[3] and i[3] not in ['1','2','3','4','5']]

[('"反应停"致"海豹儿"，该药品不良反应发生机制分型属于',
  ['C类反应（化学反应）', 'D类反应（给药反应）', 'F类反应（家庭性反应）', 'G类反应（基因毒性反应）', 'H类反应（过敏反应）'],
  ['4'],
  'B'),
 ('与硝酸甘油扩张血管作用无关的不良反应是',
  ['心率加快', '搏动性头痛', '体位性低血压', '升高眼内压', '高铁血红蛋白血症'],
  ['5'],
  'D'),
 ('秋水仙碱严重中毒的先兆反应是', ['过敏反应', '胃肠道反应', '再生障碍性贫血', '血尿', '肾衰竭'], ['2'], 'C'),
 ('渗透泵型控释制剂的推动剂是',
  ['醋酸纤维素', '乙醇', '聚羟甲基丙烯酸烷基酯', '氯化钠', 'CMC-Na溶液'],
  ['3'],
  ''),
 ('下列药物信息中属于一级信息的是', ['杂志', '药典', '摘要', '参考书', '数据库'], ['1'], '1、2'),
 ('控释小丸或膜控型片型的包衣中加入PEG的目的是', ['助悬剂', '增塑剂', '成膜剂', '乳化剂', '致孔剂'], ['5'], 'B'),
 ('分子结构中含有苯磺酰胺结构的药物是', ['唑吡坦', '硫喷妥钠', '地西泮', '氟西汀', '舒必利'], ['5'], 'B'),
 ('分子内有苯丁氧基', ['丙酸倍氯米松', '沙美特罗', '异丙托溴铵', '扎鲁司特', '茶碱'], ['2'], 'B'),
 ('需先用地塞米松及组胺H1受体阻断药预防过敏反应的药物是',
  ['塞替派', '奥沙利铂', '环磷酰胺', '丝裂霉素', '紫杉醇'],
  ['5'],
  'B'),
 ('为防止蛛网膜下腔出血患者出现脑血管痉挛，应尽早服用的药物是',
  ['钙通道阻滞剂', '钠通道阻滞剂', '钾通道阻滞剂', 'β受体阻断剂', 'M受体阻断剂'],
  ['1'],
  'A'),
 ('若要对该批产品进行取样，则应取件数为', ['10', '16', '25', '31', '35'], ['3'], '随机抽样'),
 ('经皮吸收制剂中一层不易渗透的铝塑合模的作用是',
  ['润湿皮肤促进吸收', '吸收过量的汗液',

In [61]:
len([i for i in result if i[2][0] != i[3] and i[3] not in ['1','2','3','4','5']]) / len(result)

0.21697053948564402

In [ ]:
dump_obj('result_001',result)

result_001

使用Promopt: 
你是一名医学生，正在参加考试。题目为单项选择题。参考下面例子，给出数字编号答案。
题目：书写药历是药师进行规范化药学服务的一项工作，下列内容一般不作为药历内容的是 || 选项: 1. 患者基本信息 2. 用药评价 3. 临床诊断要点 4. 用药记录 5. 患者知情同意书 || 答案: 5
题目：研究TDM的临床意义不包括 || 选项: 1. 监督临床用药 2. 研究药物在体内的代谢变化 3. 研究治疗无效的原因 4. 确定患者是否按医嘱服药 5. 研究合并用药的影响 || 答案: 3
题目："""

在该Promopt下，不用RAG， 百川2回答正确率 0.3687109019943325

错误答案中， 0.21697053948564402 是由于未能遵循指令引起的错误（回答a,b,c)

In [19]:
import re

# Open the file and read the contents
with open('book1.txt', 'r') as f:
    contents = f.readlines()

# Define the patterns to match
patterns = ['^\([一二三四五六七八九十]+\)', '^[0-9]+\.', '^\([0-9]+\)','^[一二三四五六七八九十]+']

# Loop through the contents and find the start line number of the patterns
start_line_numbers = [0]
for i, line in enumerate(contents):
    for pattern in patterns:
        if re.match(pattern, line):
            start_line_numbers.append(i)
            break

# Print the start line numbers
print(start_line_numbers)
processing_blocks = start_line_numbers[:5]

vectors_content = []

for idx in range(len(processing_blocks)):
    if idx > 0:
        ln_start = idx - 1
    else:
        ln_start = 0
    if idx > len(processing_blocks) - 2:
        ln_end =  len(processing_blocks) - 1
    else:
        ln_end = idx + 1
    print(idx, ln_start, ln_end)
    
    context_content = ''.join(contents[processing_blocks[ln_start]:processing_blocks[ln_end]]).replace('\n', '').replace(' ', '')
    print(context_content, len(context_content))
    vectors_content.append(context_content)


[0, 11, 17, 24, 41, 51, 53, 55, 58, 62, 70, 72, 74, 76, 78, 96, 99, 105, 107, 109, 111, 121, 126, 129, 132, 138, 141, 143, 164, 167, 169, 171, 172, 178, 190, 203, 205, 208, 213, 215, 218, 234, 236, 244, 254, 257, 259, 261, 266, 293, 295, 300, 303, 310, 313, 316, 319, 322, 334, 336, 338, 343, 348, 350, 352, 354, 356, 357, 358, 359, 360, 361, 363, 380, 384, 393, 395, 400, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 423, 425, 428, 433, 435, 438, 443, 445, 447, 449, 451, 454, 487, 538, 540, 549, 559, 560, 564, 583, 634, 637, 641, 647, 650, 653, 658, 660, 662, 670, 671, 674, 688, 826, 836, 838, 842, 845, 861, 875, 878, 882, 892, 900, 902, 905, 919, 1119, 1184, 1192, 1202, 1203, 1211, 1212, 1213, 1229, 1230, 1231, 1232, 1236, 1248, 1333, 1335, 1340, 1347, 1361, 1365, 1402, 1404, 1408, 1409, 1410, 1412, 1416, 1417, 1418, 1419, 1420, 1421, 1422, 1423, 1424, 1425, 1426, 1427, 1428, 1430, 1431, 1432, 1433, 1434, 1442, 1443, 1444, 1445, 1446, 1447, 1448, 1449, 1450, 1453, 1461, 1466, 1468, 

In [5]:
contents[0:17]

['\ufeff\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '现代药学的发展历程主要经历了三个阶段， 即：传统的以药品供应为中心的阶段；参与临床用药实践，促进合理用药为主的临床药学阶段；更高层次的以患者为中心，改善患者生命质量的药学服务阶段。药学服务的变化反映了现代医药学服务模式和健康理念，体现“以人为本”的宗旨，是时代进步赋予药师的使命,同时也是科学发展和药学技术进步的结果。\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '一 、 药学服务的内涵             \n',
 '\n',
 '药学服务是在临床药学工作的基础上发展 起来的，与传统的药学基础服务(供应、调 剂)有很大的区别。药学服务于1990年由美国 学者倡导，其含义是药师应用药学专业知识向 公众(包括医护人员、患者及家属)提供直接 的、负责任的、与用药相关的服务，以期提高 药物治疗的安全、有效、经济和适宜性，改善 和提高人类生活质量。\n',
 '药学服务最基本的要素是“与药物有关” 的“服务”。所谓服务，即不仅以实物形式，  还要以提供信息和知识的形式满足患者在药物  治疗上的特殊需要。药学服务中的“服务”不  同于一般行为上的功能。它包含的是药师对患者的关怀和责任，由于这种服务与药物有关， 其服务应涉及全社会所有用药的患者，包括住 院、门诊、社区和家庭患者。因此，药学服务 具有很强的社会属性。药学服务的社会属性还 表现在不仅服务于治疗性用药，还要关注预防用药和保健用药。\n',
 '药学服务在完成传统的处方调剂、药品检 验、药品供应外，更是一种更高层次的临床实 践，即必须在患者药物治疗全程中实施并获得 效果，涵盖了患者用药相关的全部需求，包括 选药、用药、疗效跟踪、用药方案与剂量调整、 不良反应规避、疾病防治和公众健康教育等。\n',
 '\n']